In [1]:
# Install the necessary modules
!pip install keras-video-generators
!pip install -q imageio
!pip install -q git+https://github.com/tensorflow/docs

  Created wheel for keras-video-generators: filename=keras_video_generators-1.0.14-py3-none-any.whl size=12882 sha256=e4c4110b835b28cdd16cbd8067dd6d58d22b6eaf4bd9780d202a282797b115cc
  Stored in directory: /root/.cache/pip/wheels/bc/22/94/eda4e8caf00c0ffd3030fecbf2e0334b8a7f038f4451e20b00
Successfully built keras-video-generators
  DEPRECATION: tensorflow-docs was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


In [2]:
# TensorFlow and TF-Hub modules.
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed

logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3

In [3]:
#@title Helper functions for the UCF101 dataset

# UCF101 will be used to evaluate our model not to train on it
# Utilities to fetch videos from UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()
# As of July 2020, crcv.ucf.edu doesn't use a certificate accepted by the
# default Colab environment anymore.
unverified_context = ssl._create_unverified_context()

def list_ucf_videos():
  """Lists videos available in UCF101 dataset."""
  global _VIDEO_LIST
  if not _VIDEO_LIST:
    index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
    videos = re.findall("(v_[\w_]+\.avi)", index)
    _VIDEO_LIST = sorted(set(videos))
  return list(_VIDEO_LIST)

def fetch_ucf_video(video):
  """Fetchs a video and cache into local filesystem."""
  cache_path = os.path.join(_CACHE_DIR, video)
  if not os.path.exists(cache_path):
    urlpath = request.urljoin(UCF_ROOT, video)
    print("Fetching %s => %s" % (urlpath, cache_path))
    data = request.urlopen(urlpath, context=unverified_context).read()
    open(cache_path, "wb").write(data)
  return cache_path

# Utilities to open video files using CV2
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(240, 240)):
  cap = cv2.VideoCapture(path)
  frames = []
  try:
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      frame = crop_center_square(frame)
      frame = cv2.resize(frame, resize)
      frame = frame[:, :, [2, 1, 0]]
      frames.append(frame)
      
      if len(frames) == max_frames:
        break
  finally:
    cap.release()
  return np.array(frames) / 255.0

def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images, fps=25)
  return embed.embed_file('./animation.gif')

In [4]:
# Load the UCF11 Dataset from the google drive
!wget  --no-check-certificate -r\
 'https://docs.google.com/uc?export=download&id=1W46sQKbcMQF3SKfQDm0Cd7jm90ap_yic' -O 'DatasetV2'

will be placed in the single file you specified.

--2021-02-25 18:14:25--  https://docs.google.com/uc?export=download&id=1W46sQKbcMQF3SKfQDm0Cd7jm90ap_yic
Resolving docs.google.com (docs.google.com)... 172.217.203.100, 172.217.203.138, 172.217.203.102, ...
Connecting to docs.google.com (docs.google.com)|172.217.203.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘DatasetV2’

DatasetV2               [ <=>                ]   3.20K  --.-KB/s    in 0s      

2021-02-25 18:14:25 (35.8 MB/s) - ‘DatasetV2’ saved [3275]

Loading robots.txt; please ignore errors.
--2021-02-25 18:14:25--  https://docs.google.com/robots.txt
Reusing existing connection to docs.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘DatasetV2’

DatasetV2               [ <=>                ]     567  --.-KB/s    in 0s      

2021-02-25 18:14:26 (23.3 MB/s) - ‘DatasetV2’ saved [567]

--2021-02-25 18:14

In [5]:
# Unzip the UCF11 dataset into 1 folder with 2 subfolders train and test
!unzip DatasetV2

Archive:  DatasetV2
warning [DatasetV2]:  315129 extra bytes at beginning or within zipfile
  (attempting to process anyway)
   creating: DataSetV2/
   creating: DataSetV2/test/
   creating: DataSetV2/test/basketball/
  inflating: DataSetV2/test/basketball/v_shooting_20_01.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_20_02.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_20_03.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_20_04.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_20_05.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_20_06.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_20_07.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_21_01.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_21_02.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_21_03.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_21_04.mpg  
  inflating: DataSetV2/test/basketball/v_shooting_21_05.mpg  
  inflating: DataSetV2/test/basketball

  inflating: DataSetV2/test/golf_swing/v_golf_23_01.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_23_02.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_23_03.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_23_04.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_23_05.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_23_06.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_23_07.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_23_08.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_24_01.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_24_02.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_24_03.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_24_04.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_24_05.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_24_06.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_24_07.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_25_01.mpg  
  inflating: DataSetV2/test/golf_swing/v_golf_25_02.mpg  
  inflating: D

  inflating: DataSetV2/test/swing/v_swing_22_05.mpg  
  inflating: DataSetV2/test/swing/v_swing_23_01.mpg  
  inflating: DataSetV2/test/swing/v_swing_23_02.mpg  
  inflating: DataSetV2/test/swing/v_swing_23_03.mpg  
  inflating: DataSetV2/test/swing/v_swing_23_04.mpg  
  inflating: DataSetV2/test/swing/v_swing_23_05.mpg  
  inflating: DataSetV2/test/swing/v_swing_24_01.mpg  
  inflating: DataSetV2/test/swing/v_swing_24_02.mpg  
  inflating: DataSetV2/test/swing/v_swing_24_03.mpg  
  inflating: DataSetV2/test/swing/v_swing_24_04.mpg  
  inflating: DataSetV2/test/swing/v_swing_24_05.mpg  
  inflating: DataSetV2/test/swing/v_swing_25_01.mpg  
  inflating: DataSetV2/test/swing/v_swing_25_02.mpg  
  inflating: DataSetV2/test/swing/v_swing_25_03.mpg  
  inflating: DataSetV2/test/swing/v_swing_25_04.mpg  
   creating: DataSetV2/test/tennis_swing/
  inflating: DataSetV2/test/tennis_swing/v_tennis_20_02.mpg  
  inflating: DataSetV2/test/tennis_swing/v_tennis_20_03.mpg  
  inflating: DataSetV2/t

  inflating: DataSetV2/test/walking/v_walk_dog_24_06.mpg  
  inflating: DataSetV2/test/walking/v_walk_dog_25_01.mpg  
  inflating: DataSetV2/test/walking/v_walk_dog_25_02.mpg  
  inflating: DataSetV2/test/walking/v_walk_dog_25_03.mpg  
  inflating: DataSetV2/test/walking/v_walk_dog_25_04.mpg  
   creating: DataSetV2/train/
   creating: DataSetV2/train/basketball/
  inflating: DataSetV2/train/basketball/v_shooting_01_01.mpg  
  inflating: DataSetV2/train/basketball/v_shooting_01_02.mpg  
  inflating: DataSetV2/train/basketball/v_shooting_01_03.mpg  
  inflating: DataSetV2/train/basketball/v_shooting_01_04.mpg  
  inflating: DataSetV2/train/basketball/v_shooting_01_05.mpg  
  inflating: DataSetV2/train/basketball/v_shooting_01_06.mpg  
  inflating: DataSetV2/train/basketball/v_shooting_01_07.mpg  
  inflating: DataSetV2/train/basketball/v_shooting_02_01.mpg  
  inflating: DataSetV2/train/basketball/v_shooting_02_02.mpg  
  inflating: DataSetV2/train/basketball/v_shooting_02_03.mpg  
  in

  inflating: DataSetV2/train/biking/v_biking_05_09.mpg  
  inflating: DataSetV2/train/biking/v_biking_06_01.mpg  
  inflating: DataSetV2/train/biking/v_biking_06_02.mpg  
  inflating: DataSetV2/train/biking/v_biking_06_03.mpg  
  inflating: DataSetV2/train/biking/v_biking_06_04.mpg  
  inflating: DataSetV2/train/biking/v_biking_06_05.mpg  
  inflating: DataSetV2/train/biking/v_biking_07_01.mpg  
  inflating: DataSetV2/train/biking/v_biking_07_02.mpg  
  inflating: DataSetV2/train/biking/v_biking_07_03.mpg  
  inflating: DataSetV2/train/biking/v_biking_07_04.mpg  
  inflating: DataSetV2/train/biking/v_biking_07_05.mpg  
  inflating: DataSetV2/train/biking/v_biking_07_06.mpg  
  inflating: DataSetV2/train/biking/v_biking_08_01.mpg  
  inflating: DataSetV2/train/biking/v_biking_08_02.mpg  
  inflating: DataSetV2/train/biking/v_biking_08_03.mpg  
  inflating: DataSetV2/train/biking/v_biking_08_04.mpg  
  inflating: DataSetV2/train/biking/v_biking_08_05.mpg  
  inflating: DataSetV2/train/bi

...

  inflating: DataSetV2/train/soccer_juggling/v_juggle_17_06.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_17_07.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_18_01.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_18_02.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_18_03.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_18_04.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_18_05.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_18_06.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_18_07.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_18_08.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_19_01.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_19_02.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_19_03.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_19_04.mpg  
  inflating: DataSetV2/train/soccer_juggling/v_juggle_19_05.mpg  
  inflatin

  inflating: DataSetV2/train/tennis_swing/v_tennis_04_04.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_04_05.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_04_06.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_04_07.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_05_01.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_05_02.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_05_03.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_05_04.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_05_05.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_05_06.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_05_07.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_06_01.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_06_02.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_06_03.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_06_04.mpg  
  inflating: DataSetV2/train/tennis_swing/v_tennis_06_0

  inflating: DataSetV2/train/trampoline_jumping/v_jumping_06_03.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_06_04.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_07_01.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_07_02.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_07_03.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_07_04.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_07_05.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_08_01.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_08_02.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_08_03.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_08_04.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_09_01.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_09_02.mpg  
  inflating: DataSetV2/train/trampoline_jumping/v_jumping_09_03.mpg  
  inflating: DataSet

  inflating: DataSetV2/train/volleyball_spiking/v_spiking_11_01.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_11_02.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_11_03.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_11_04.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_11_05.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_11_06.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_12_01.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_12_02.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_12_03.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_12_04.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_13_01.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_13_02.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_13_03.mpg  
  inflating: DataSetV2/train/volleyball_spiking/v_spiking_13_04.mpg  
  inflating: DataSet

  inflating: DataSetV2/train/walking/v_walk_dog_22_01.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_22_02.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_22_03.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_23_04.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_24_01.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_24_02.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_24_03.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_24_04.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_24_05.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_24_06.mpg  
  inflating: DataSetV2/train/walking/v_walk_dog_25_01.mpg  


In [6]:
import os
import glob
import keras
from keras_video import VideoFrameGenerator
# use sub directories names as classes
classes = [i.split(os.path.sep)[3] for i in glob.glob('./DataSetV2/train/*')]
classes.sort()

In [7]:
classes

['basketball',
 'biking',
 'diving',
 'golf_swing',
 'horse_riding',
 'soccer_juggling',
 'swing',
 'tennis_swing',
 'trampoline_jumping',
 'volleyball_spiking',
 'walking']

In [8]:
#@title Make all the files in 1 subfolder for each class
# import shutil

# for Class in classes:
#   subfolder = [j.split(os.path.sep)[4] for j in glob.glob(f'/content/UCF11_updated_mpg/{Class}/*')]
#   subfolder.sort()
#   for i in range(1,26):
#     folder = [j for j in glob.glob(f'/content/UCF11_updated_mpg/{Class}/{subfolder[i]}/*')]
#     if subfolder[i-1][0] == 'v':
#       for File in folder:
#         dst = f'/content/UCF11_updated_mpg/{Class}/'
#         shutil.move(File, dst)
#shutil.make_archive('DatasetV2', 'zip', 'UCF11_updated_mpg')

In [39]:
# some global params
SIZE = (240, 240)
CHANNELS = 3
NBFRAME = 5 
BS = 8
# pattern to get videos and classes
glob_pattern= './DataSetV2/train/{classname}/*.mpg'
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip= True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.1, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

class basketball, validation count: 9, train count: 90
class biking, validation count: 10, train count: 92
class diving, validation count: 11, train count: 108
class golf_swing, validation count: 10, train count: 91
class horse_riding, validation count: 15, train count: 141
class soccer_juggling, validation count: 12, train count: 108
class swing, validation count: 10, train count: 98
class tennis_swing, validation count: 13, train count: 119
class trampoline_jumping, validation count: 9, train count: 87
class volleyball_spiking, validation count: 8, train count: 80
class walking, validation count: 10, train count: 94
Total data: 11 classes for 1108 files for train


In [40]:
valid = train.get_validation_generator()

Total data: 11 classes for 117 files for validation


In [41]:
import keras_video.utils
# keras_video.utils.show_sample(train)

In [42]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization,                                    AveragePooling2D,experimental,Input, GlobalMaxPooling2D
from tensorflow.keras.layers import Lambda, Activation, concatenate
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.applications import Xception

In [43]:
base_model = Xception(include_top=False, weights='imagenet',\
                                         input_shape=(240,240,3), classes=len(classes))

for layer in base_model.layers:
    layer.trainable = False

In [44]:
from tensorflow.keras import layers

for layer in base_model.layers[-4:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

In [45]:
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 240, 240, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 119, 119, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 119, 119, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 119, 119, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [46]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import TimeDistributed, LSTM, Flatten
model = Sequential()
# add Xception model for 5 input images (keeping the right shape
model.add(
    TimeDistributed(base_model, input_shape=(5, 240, 240, 3))
)
# now, flatten on each output to send 5 
# outputs with one dimension to LSTM
model.add(
    TimeDistributed(
        Flatten()
    )
)
model.add(LSTM(64, activation='relu', return_sequences=False))
# finalize with standard Dense, Dropout...
model.add(Dense(1024, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(11, activation='softmax'))

In [47]:
import tensorflow
opt = tensorflow.optimizers.Adam(learning_rate=0.001)

In [48]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stopping = EarlyStopping(monitor = 'val_acc', patience= 7, restore_best_weights = True)

lr_scheduler = ReduceLROnPlateau(monitor = 'val_acc', patience = 3, factor = 0.333)

In [49]:
model.compile(opt, 'categorical_crossentropy', metrics=['acc'])

In [50]:
tensorflow.config.run_functions_eagerly(False)

In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 5, 8, 8, 2048)     20861480  
_________________________________________________________________
time_distributed_3 (TimeDist (None, 5, 131072)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33571072  
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              66560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                65600     
_________________________________________________________________
dense_5 (Dense)              (None, 11)               

In [52]:
model.fit(
    train,
    validation_data=valid,
    verbose=1,
    epochs=30,
    callbacks = [early_stopping, lr_scheduler]
)

Epoch 1/30
138/138 [==============================] - 127s 894ms/step - loss: 4.3810 - acc: 0.2920 - val_loss: 0.9937 - val_acc: 0.6696
Epoch 2/30
138/138 [==============================] - 70s 504ms/step - loss: 1.0979 - acc: 0.6838 - val_loss: 0.4216 - val_acc: 0.8482
Epoch 3/30
138/138 [==============================] - 70s 506ms/step - loss: 0.9354 - acc: 0.7480 - val_loss: 0.8714 - val_acc: 0.7500
Epoch 4/30
138/138 [==============================] - 70s 505ms/step - loss: 0.8446 - acc: 0.7728 - val_loss: 0.5694 - val_acc: 0.8482
Epoch 5/30
138/138 [==============================] - 70s 507ms/step - loss: 0.5892 - acc: 0.8557 - val_loss: 0.3855 - val_acc: 0.8929
Epoch 6/30
138/138 [==============================] - 68s 495ms/step - loss: 0.3308 - acc: 0.8991 - val_loss: 0.2201 - val_acc: 0.9375
Epoch 7/30
138/138 [==============================] - 70s 503ms/step - loss: 0.3211 - acc: 0.9145 - val_loss: 0.9269 - val_acc: 0.8750
Epoch 8/30
138/138 [==============================] - 

In [23]:
# pattern to get test videos and classes
glob_pattern= './DataSetV2/test/{classname}/*.mpg'

# Create video frame generator for testing
test = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME, 
    shuffle=True,
    batch_size=1,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=None,
    use_frame_cache=False)

Total data: 11 classes for 404 files for train


In [53]:
model.evaluate(test)

404/404 [==============================] - 8s 21ms/step - loss: 0.8331 - acc: 0.8787


[0.8330821990966797, 0.8787128925323486]

In [54]:
ucf_videos = list_ucf_videos()
categories = {}
for video in ucf_videos:
  category = video[2:-12]
  if category not in categories:
    categories[category] = []
  categories[category].append(video)
print("Found %d videos in %d categories." % (len(ucf_videos), len(categories)))
for category, sequences in categories.items():
  summary = ", ".join(sequences[:2])
  print("%-20s %4d videos (%s, ...)" % (category, len(sequences), summary))

Found 13320 videos in 101 categories.
ApplyEyeMakeup        145 videos (v_ApplyEyeMakeup_g01_c01.avi, v_ApplyEyeMakeup_g01_c02.avi, ...)
ApplyLipstick         114 videos (v_ApplyLipstick_g01_c01.avi, v_ApplyLipstick_g01_c02.avi, ...)
Archery               145 videos (v_Archery_g01_c01.avi, v_Archery_g01_c02.avi, ...)
BabyCrawling          132 videos (v_BabyCrawling_g01_c01.avi, v_BabyCrawling_g01_c02.avi, ...)
BalanceBeam           108 videos (v_BalanceBeam_g01_c01.avi, v_BalanceBeam_g01_c02.avi, ...)
BandMarching          155 videos (v_BandMarching_g01_c01.avi, v_BandMarching_g01_c02.avi, ...)
BaseballPitch         150 videos (v_BaseballPitch_g01_c01.avi, v_BaseballPitch_g01_c02.avi, ...)
BasketballDunk        131 videos (v_BasketballDunk_g01_c01.avi, v_BasketballDunk_g01_c02.avi, ...)
Basketball            134 videos (v_Basketball_g01_c01.avi, v_Basketball_g01_c02.avi, ...)
BenchPress            160 videos (v_BenchPress_g01_c01.avi, v_BenchPress_g01_c02.avi, ...)
Biking              

In [60]:
video_path = fetch_ucf_video("v_WalkingWithDog_g01_c02.avi")
sample_video = load_video(video_path, 30)

In [61]:
sample_video = sample_video.reshape(1,-1,240,240,3)
sample_video.shape

(1, 30, 240, 240, 3)

In [62]:
x = model.predict(sample_video)

In [63]:
x

array([[0.0000000e+00, 1.3675132e-34, 0.0000000e+00, 0.0000000e+00,
        4.8147445e-13, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 1.0000000e+00]], dtype=float32)

In [64]:
print(np.argmax(x, axis=1))

[10]


In [65]:
classes

['basketball',
 'biking',
 'diving',
 'golf_swing',
 'horse_riding',
 'soccer_juggling',
 'swing',
 'tennis_swing',
 'trampoline_jumping',
 'volleyball_spiking',
 'walking']

In [66]:
model.save('Xception.h5')